In [1]:
import cv2
import os

def concatenate_videos(video1_path, video2_path, output_path):
    # Open the video files
    cap1 = cv2.VideoCapture(video1_path)
    cap2 = cv2.VideoCapture(video2_path)

    # Get properties of the first video
    width1 = int(cap1.get(cv2.CAP_PROP_FRAME_WIDTH))
    height1 = int(cap1.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps1 = cap1.get(cv2.CAP_PROP_FPS)
    frame_count1 = int(cap1.get(cv2.CAP_PROP_FRAME_COUNT))

    # Get properties of the second video
    width2 = int(cap2.get(cv2.CAP_PROP_FRAME_WIDTH))
    height2 = int(cap2.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps2 = cap2.get(cv2.CAP_PROP_FPS)
    frame_count2 = int(cap2.get(cv2.CAP_PROP_FRAME_COUNT))

    # Ensure both videos have the same frame rate
    if fps1 != fps2:
        raise ValueError("The two videos have different frame rates.")

    # Determine the minimum frame count
    min_frame_count = min(frame_count1, frame_count2)

    # Determine the new dimensions (use the smaller height)
    new_height = min(height1, height2)
    new_width1 = int(width1 * (new_height / height1))
    new_width2 = int(width2 * (new_height / height2))

    # Define the codec and create VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps1, (new_width1 + new_width2, new_height))

    # Loop through each frame
    for i in range(min_frame_count):
        ret1, frame1 = cap1.read()
        ret2, frame2 = cap2.read()

        if not ret1 or not ret2:
            break

        # Resize frames
        frame1_resized = cv2.resize(frame1, (new_width1, new_height))
        frame2_resized = cv2.resize(frame2, (new_width2, new_height))

        # Concatenate frames horizontally
        concatenated_frame = cv2.hconcat([frame1_resized, frame2_resized])

        # Write the frame to the output video
        out.write(concatenated_frame)

    # Release everything if job is finished
    cap1.release()
    cap2.release()
    out.release()



In [6]:
# Ensure the videos directory exists
os.makedirs("./videos", exist_ok=True)

# Paths to input videos
video1_path = "/home/sysadm/Downloads/emotalkingface/videos/last_time/tr_train_result_neu_kaggle_2_im01_check/neutral_generated.mp4_.mp4"
video2_path = "/home/sysadm/Downloads/emotalkingface/videos/last_time/pre_train_result_neu_kaggle_2_im01_check/neutral_generated.mp4_.mp4"

# Path to output video
output_video_path = "./videos/kaggle_1.mp4"

# Concatenate videos
concatenate_videos(video1_path, video2_path, output_video_path)

print(f"Output video saved to {output_video_path}")

Output video saved to ./videos/kaggle_1.mp4


In [11]:
# Ensure the videos directory exists
os.makedirs("./videos", exist_ok=True)

# Paths to input videos
video1_path = "./videos/video_1.mp4"
video2_path = "./videos/video_2.mp4"

# Path to output video
output_video_path = "./videos/side_by_side_output.mp4"

# Concatenate videos
concatenate_videos(video1_path, video2_path, output_video_path)

print(f"Output video saved to {output_video_path}")

Output video saved to ./videos/side_by_side_output.mp4


In [ ]:
#PSNR

In [8]:
import cv2
import numpy as np

def calculate_psnr(img1, img2):
    mse = np.mean((img1 - img2) ** 2)
    if mse == 0:
        return float('inf')
    psnr = 20 * np.log10(255.0 / np.sqrt(mse))
    return psnr

def get_video_frames(video_path):
    cap = cv2.VideoCapture(video_path)
    frames = []
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frames.append(frame)
    cap.release()
    return frames

def calculate_video_psnr(video_path1, video_path2):
    frames1 = get_video_frames(video_path1)
    frames2 = get_video_frames(video_path2)
    
    if len(frames1) != len(frames2):
        raise ValueError("Videos do not have the same number of frames")

    total_psnr = 0
    frame_count = len(frames1)
    
    for frame1, frame2 in zip(frames1, frames2):
        psnr = calculate_psnr(frame1, frame2)
        total_psnr += psnr
    
    avg_psnr = total_psnr / frame_count
    return avg_psnr



In [10]:
# Paths to the generated and ground-truth video files
generated_video_path = "./videos/video_1.mp4"
pre_train_video_path = "./videos/video_2.mp4"

# Calculate average PSNR for the video
avg_psnr = calculate_video_psnr(generated_video_path, pre_train_video_path)
print(f"Average PSNR: {avg_psnr} dB")

Average PSNR: 31.87061262602449 dB


In [ ]:
#SSIM

In [14]:
from skimage.metrics import structural_similarity as ssim
import cv2

from skimage.metrics import structural_similarity as ssim
import cv2

def calculate_ssim(img1, img2):
    # Convert the images to grayscale (SSIM is often computed on grayscale images)
    gray_img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
    gray_img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)
    
    # Compute SSIM
    score, _ = ssim(gray_img1, gray_img2, full=True)
    return score
    
def get_video_frames(video_path):
    cap = cv2.VideoCapture(video_path)
    frames = []
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frames.append(frame)
    cap.release()
    return frames

def calculate_video_ssim(video_path1, video_path2):
    frames1 = get_video_frames(video_path1)
    frames2 = get_video_frames(video_path2)
    
    if len(frames1) != len(frames2):
        raise ValueError("Videos do not have the same number of frames")

    total_ssim = 0
    frame_count = len(frames1)
    
    for frame1, frame2 in zip(frames1, frames2):
        ssim_score = calculate_ssim(frame1, frame2)
        total_ssim += ssim_score
    
    avg_ssim = total_ssim / frame_count
    return avg_ssim



In [15]:
# Paths to the generated and ground-truth video files
generated_video_path = "./videos/video_1.mp4"
pre_train_video_path = "./videos/video_2.mp4"

# Calculate average SSIM for the video
avg_ssim = calculate_video_ssim(generated_video_path, pre_train_video_path)
print(f"Average SSIM: {avg_ssim}")

Average SSIM: 0.7730546636612264


In [16]:
# Paths to input videos
video1_path = "/home/sysadm/Downloads/emotalkingface/videos/last_time/tr_train_result_neu_kaggle_2_im01_check/neutral_generated.mp4_.mp4"
video2_path = "/home/sysadm/Downloads/emotalkingface/videos/last_time/pre_train_result_neu_kaggle_2_im01_check/neutral_generated.mp4_.mp4"

In [17]:
# Paths to the generated and ground-truth video files
generated_video_path = video1_path
pre_train_video_path = video2_path

# Calculate average PSNR for the video
avg_psnr = calculate_video_psnr(generated_video_path, pre_train_video_path)
print(f"Average PSNR: {avg_psnr} dB")

Average PSNR: 37.554057157103884 dB


In [19]:
# Paths to input videos
video1_path = "/home/sysadm/Downloads/emotalkingface/videos/last_time/tr_train_result_neu_kaggle_im01_check/neutral_generated.mp4_.mp4"
video2_path = "/home/sysadm/Downloads/emotalkingface/videos/last_time/pre_train_result_neu_kaggle_im01_check/neutral_generated.mp4_.mp4"

In [20]:
# Paths to the generated and ground-truth video files
generated_video_path = video1_path
pre_train_video_path = video2_path

# Calculate average PSNR for the video
avg_psnr = calculate_video_psnr(generated_video_path, pre_train_video_path)
print(f"Average PSNR: {avg_psnr} dB")

Average PSNR: 35.993394387174426 dB


In [ ]:
#SSIM

In [21]:
# Paths to the generated and ground-truth video files
generated_video_path = video1_path
pre_train_video_path = video2_path

# Calculate average SSIM for the video
avg_ssim = calculate_video_ssim(generated_video_path, pre_train_video_path)
print(f"Average SSIM: {avg_ssim}")

Average SSIM: 0.9232746214177845


In [ ]:
#Random Image

In [ ]:
#video concat

In [25]:
# Ensure the videos directory exists
os.makedirs("./videos", exist_ok=True)

# Paths to input videos
video1_path = "/home/sysadm/Downloads/emotalkingface/videos/last_time/tr_train_result_neu_soumitra_sir_1_check/neutral_generated.mp4_.mp4"
video2_path = "/home/sysadm/Downloads/emotalkingface/videos/last_time/pre_train_result_neu_soumitra_sir_1_check/neutral_generated.mp4_.mp4"

# Path to output video
output_video_path = "./videos/sou_sir_1.mp4"

# Concatenate videos
concatenate_videos(video1_path, video2_path, output_video_path)

print(f"Output video saved to {output_video_path}")

Output video saved to ./videos/sou_sir_1.mp4


In [ ]:
#Calculate psnr

In [26]:
# Paths to input videos
video1_path = "/home/sysadm/Downloads/emotalkingface/videos/last_time/tr_train_result_neu_soumitra_sir_1_check/neutral_generated.mp4_.mp4"
video2_path = "/home/sysadm/Downloads/emotalkingface/videos/last_time/pre_train_result_neu_soumitra_sir_1_check/neutral_generated.mp4_.mp4"


In [27]:
# Paths to the generated and ground-truth video files
generated_video_path = video1_path
pre_train_video_path = video2_path

# Calculate average PSNR for the video
avg_psnr = calculate_video_psnr(generated_video_path, pre_train_video_path)
print(f"Average PSNR: {avg_psnr} dB")

Average PSNR: 34.37892241345245 dB


In [ ]:
#SSIM

In [28]:
# Paths to the generated and ground-truth video files
generated_video_path = video1_path
pre_train_video_path = video2_path

# Calculate average SSIM for the video
avg_ssim = calculate_video_ssim(generated_video_path, pre_train_video_path)
print(f"Average SSIM: {avg_ssim}")

Average SSIM: 0.9479626391590802


In [29]:
# Paths to input videos
video1_path = "/home/sysadm/Downloads/emotalkingface/videos/last_time/tr_train_result_neu_soumitra_sir_2_check/neutral_generated.mp4_.mp4"
video2_path = "/home/sysadm/Downloads/emotalkingface/videos/last_time/pre_train_result_neu_soumitra_sir_2_check/neutral_generated.mp4_.mp4"


In [30]:
# Paths to the generated and ground-truth video files
generated_video_path = video1_path
pre_train_video_path = video2_path

# Calculate average PSNR for the video
avg_psnr = calculate_video_psnr(generated_video_path, pre_train_video_path)
print(f"Average PSNR: {avg_psnr} dB")

Average PSNR: 34.56392738415843 dB


In [31]:
# Paths to the generated and ground-truth video files
generated_video_path = video1_path
pre_train_video_path = video2_path

# Calculate average SSIM for the video
avg_ssim = calculate_video_ssim(generated_video_path, pre_train_video_path)
print(f"Average SSIM: {avg_ssim}")

Average SSIM: 0.9571074464239112


In [1]:
import os
import shutil

# Define the emotion and intensity dictionaries
emotion_dict = {'ANG': 0, 'DIS': 1, 'FEA': 2, 'HAP': 3, 'NEU': 4, 'SAD': 5}
intensity_dict = {'XX': 0, 'LO': 1, 'MD': 2, 'HI': 3}

def copy_neu_emotion_videos(input_folder, output_folder):
    """
    Copy files with 'NEU' emotion from input_folder to output_folder.
    
    Parameters:
    - input_folder: str, path to the input folder containing video files
    - output_folder: str, path to the output folder where 'NEU' emotion videos will be copied
    """
    # Create the output folder if it doesn't exist
    os.makedirs(output_folder, exist_ok=True)

    # Iterate through the files in the input folder
    for root, dirnames, filenames in os.walk(input_folder):
        for filename in filenames:
            if os.path.splitext(filename)[1] == '.mp4':
                labels = os.path.splitext(filename)[0].split('_')
                emotion = labels[2]
                
                # Check if the emotion is 'NEU'
                if emotion == 'NEU':
                    file_path = os.path.join(root, filename)
                    output_path = os.path.join(output_folder, filename)
                    
                    # Copy the file to the output folder
                    shutil.copy(file_path, output_path)
                    print("File with emotion 'NEU' copied to:", output_path)

In [3]:
# Example usage:
input_folder = '/home/sysadm/Downloads/emotalkingface/AudioMP4/'
output_folder ='/home/sysadm/Downloads/emotalkingface/NEU_vD'
copy_neu_emotion_videos(input_folder, output_folder)

File with emotion 'NEU' copied to: /home/sysadm/Downloads/emotalkingface/NEU_vD/1005_IEO_NEU_XX.mp4
File with emotion 'NEU' copied to: /home/sysadm/Downloads/emotalkingface/NEU_vD/1076_TAI_NEU_XX.mp4
File with emotion 'NEU' copied to: /home/sysadm/Downloads/emotalkingface/NEU_vD/1015_DFA_NEU_XX.mp4
File with emotion 'NEU' copied to: /home/sysadm/Downloads/emotalkingface/NEU_vD/1044_DFA_NEU_XX.mp4
File with emotion 'NEU' copied to: /home/sysadm/Downloads/emotalkingface/NEU_vD/1015_MTI_NEU_XX.mp4
File with emotion 'NEU' copied to: /home/sysadm/Downloads/emotalkingface/NEU_vD/1049_ITH_NEU_XX.mp4
File with emotion 'NEU' copied to: /home/sysadm/Downloads/emotalkingface/NEU_vD/1001_MTI_NEU_XX.mp4
File with emotion 'NEU' copied to: /home/sysadm/Downloads/emotalkingface/NEU_vD/1065_IWL_NEU_XX.mp4
File with emotion 'NEU' copied to: /home/sysadm/Downloads/emotalkingface/NEU_vD/1025_IOM_NEU_XX.mp4
File with emotion 'NEU' copied to: /home/sysadm/Downloads/emotalkingface/NEU_vD/1031_ITS_NEU_XX.mp4


File with emotion 'NEU' copied to: /home/sysadm/Downloads/emotalkingface/NEU_vD/1057_DFA_NEU_XX.mp4
File with emotion 'NEU' copied to: /home/sysadm/Downloads/emotalkingface/NEU_vD/1001_IEO_NEU_XX.mp4
File with emotion 'NEU' copied to: /home/sysadm/Downloads/emotalkingface/NEU_vD/1035_ITH_NEU_XX.mp4
File with emotion 'NEU' copied to: /home/sysadm/Downloads/emotalkingface/NEU_vD/1028_DFA_NEU_XX.mp4
File with emotion 'NEU' copied to: /home/sysadm/Downloads/emotalkingface/NEU_vD/1061_MTI_NEU_XX.mp4
File with emotion 'NEU' copied to: /home/sysadm/Downloads/emotalkingface/NEU_vD/1046_IWL_NEU_XX.mp4
File with emotion 'NEU' copied to: /home/sysadm/Downloads/emotalkingface/NEU_vD/1081_IWW_NEU_XX.mp4
File with emotion 'NEU' copied to: /home/sysadm/Downloads/emotalkingface/NEU_vD/1066_DFA_NEU_XX.mp4
File with emotion 'NEU' copied to: /home/sysadm/Downloads/emotalkingface/NEU_vD/1063_WSI_NEU_XX.mp4
File with emotion 'NEU' copied to: /home/sysadm/Downloads/emotalkingface/NEU_vD/1002_IOM_NEU_XX.mp4


File with emotion 'NEU' copied to: /home/sysadm/Downloads/emotalkingface/NEU_vD/1005_TAI_NEU_XX.mp4
File with emotion 'NEU' copied to: /home/sysadm/Downloads/emotalkingface/NEU_vD/1025_IWW_NEU_XX.mp4
File with emotion 'NEU' copied to: /home/sysadm/Downloads/emotalkingface/NEU_vD/1031_TIE_NEU_XX.mp4
File with emotion 'NEU' copied to: /home/sysadm/Downloads/emotalkingface/NEU_vD/1028_TIE_NEU_XX.mp4
File with emotion 'NEU' copied to: /home/sysadm/Downloads/emotalkingface/NEU_vD/1078_MTI_NEU_XX.mp4
File with emotion 'NEU' copied to: /home/sysadm/Downloads/emotalkingface/NEU_vD/1023_WSI_NEU_XX.mp4
File with emotion 'NEU' copied to: /home/sysadm/Downloads/emotalkingface/NEU_vD/1027_DFA_NEU_XX.mp4
File with emotion 'NEU' copied to: /home/sysadm/Downloads/emotalkingface/NEU_vD/1048_IWL_NEU_XX.mp4
File with emotion 'NEU' copied to: /home/sysadm/Downloads/emotalkingface/NEU_vD/1043_WSI_NEU_XX.mp4
File with emotion 'NEU' copied to: /home/sysadm/Downloads/emotalkingface/NEU_vD/1015_TIE_NEU_XX.mp4


File with emotion 'NEU' copied to: /home/sysadm/Downloads/emotalkingface/NEU_vD/1025_ITH_NEU_XX.mp4
File with emotion 'NEU' copied to: /home/sysadm/Downloads/emotalkingface/NEU_vD/1004_TIE_NEU_XX.mp4
File with emotion 'NEU' copied to: /home/sysadm/Downloads/emotalkingface/NEU_vD/1070_IOM_NEU_XX.mp4
File with emotion 'NEU' copied to: /home/sysadm/Downloads/emotalkingface/NEU_vD/1072_WSI_NEU_XX.mp4
File with emotion 'NEU' copied to: /home/sysadm/Downloads/emotalkingface/NEU_vD/1075_TAI_NEU_XX.mp4
File with emotion 'NEU' copied to: /home/sysadm/Downloads/emotalkingface/NEU_vD/1018_IEO_NEU_XX.mp4
File with emotion 'NEU' copied to: /home/sysadm/Downloads/emotalkingface/NEU_vD/1035_TIE_NEU_XX.mp4
File with emotion 'NEU' copied to: /home/sysadm/Downloads/emotalkingface/NEU_vD/1060_DFA_NEU_XX.mp4
File with emotion 'NEU' copied to: /home/sysadm/Downloads/emotalkingface/NEU_vD/1055_DFA_NEU_XX.mp4
File with emotion 'NEU' copied to: /home/sysadm/Downloads/emotalkingface/NEU_vD/1039_TAI_NEU_XX.mp4


File with emotion 'NEU' copied to: /home/sysadm/Downloads/emotalkingface/NEU_vD/1008_IEO_NEU_XX.mp4
File with emotion 'NEU' copied to: /home/sysadm/Downloads/emotalkingface/NEU_vD/1065_IEO_NEU_XX.mp4
File with emotion 'NEU' copied to: /home/sysadm/Downloads/emotalkingface/NEU_vD/1008_ITH_NEU_XX.mp4
File with emotion 'NEU' copied to: /home/sysadm/Downloads/emotalkingface/NEU_vD/1059_DFA_NEU_XX.mp4
File with emotion 'NEU' copied to: /home/sysadm/Downloads/emotalkingface/NEU_vD/1046_IWW_NEU_XX.mp4
File with emotion 'NEU' copied to: /home/sysadm/Downloads/emotalkingface/NEU_vD/1011_IWL_NEU_XX.mp4
File with emotion 'NEU' copied to: /home/sysadm/Downloads/emotalkingface/NEU_vD/1076_IOM_NEU_XX.mp4
File with emotion 'NEU' copied to: /home/sysadm/Downloads/emotalkingface/NEU_vD/1052_IOM_NEU_XX.mp4
File with emotion 'NEU' copied to: /home/sysadm/Downloads/emotalkingface/NEU_vD/1050_TAI_NEU_XX.mp4
File with emotion 'NEU' copied to: /home/sysadm/Downloads/emotalkingface/NEU_vD/1065_MTI_NEU_XX.mp4


File with emotion 'NEU' copied to: /home/sysadm/Downloads/emotalkingface/NEU_vD/1029_DFA_NEU_XX.mp4
File with emotion 'NEU' copied to: /home/sysadm/Downloads/emotalkingface/NEU_vD/1054_IOM_NEU_XX.mp4
File with emotion 'NEU' copied to: /home/sysadm/Downloads/emotalkingface/NEU_vD/1037_MTI_NEU_XX.mp4
File with emotion 'NEU' copied to: /home/sysadm/Downloads/emotalkingface/NEU_vD/1062_DFA_NEU_XX.mp4
File with emotion 'NEU' copied to: /home/sysadm/Downloads/emotalkingface/NEU_vD/1085_MTI_NEU_XX.mp4
File with emotion 'NEU' copied to: /home/sysadm/Downloads/emotalkingface/NEU_vD/1052_MTI_NEU_XX.mp4
File with emotion 'NEU' copied to: /home/sysadm/Downloads/emotalkingface/NEU_vD/1026_IOM_NEU_XX.mp4
File with emotion 'NEU' copied to: /home/sysadm/Downloads/emotalkingface/NEU_vD/1058_TAI_NEU_XX.mp4
File with emotion 'NEU' copied to: /home/sysadm/Downloads/emotalkingface/NEU_vD/1034_TSI_NEU_XX.mp4
File with emotion 'NEU' copied to: /home/sysadm/Downloads/emotalkingface/NEU_vD/1007_WSI_NEU_XX.mp4


In [4]:
import cv2
import os

def save_first_frame(video_path, output_image_path):
    # Ensure the output directory exists
    output_dir = os.path.dirname(output_image_path)
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    # Open the video file
    cap = cv2.VideoCapture(video_path)
    
    # Check if video opened successfully
    if not cap.isOpened():
        print(f"Error: Cannot open video file {video_path}")
        return
    
    # Read the first frame
    ret, frame = cap.read()
    
    # Check if frame is read correctly
    if ret:
        # Save the first frame as an image
        cv2.imwrite(output_image_path, frame)
        print(f"First frame saved to {output_image_path}")
    else:
        print("Error: Cannot read the first frame from the video")
    
    # Release the video capture object
    cap.release()
# Example usage
video_path = '/home/sysadm/Downloads/emotalkingface/NEU_vD/1001_DFA_NEU_XX.mp4'
output_image_path = '/home/sysadm/Downloads/emotalkingface/first_frame_image/first_frame.jpg'
save_first_frame(video_path, output_image_path)


First frame saved to /home/sysadm/Downloads/emotalkingface/first_frame_image/first_frame.jpg


In [10]:
import dlib
import cv2
import numpy as np

# Load pre-trained face detector and landmark predictor from dlib
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')  # You need to download this file

def extract_landmarks(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    faces = detector(gray)
    if len(faces) == 0:
        return None  # No faces detected
    landmarks = predictor(gray, faces[0])
    landmarks = [(p.x, p.y) for p in landmarks.parts()]
    return np.array(landmarks)

def calculate_nlmd(landmarks1, landmarks2):
    if landmarks1 is None or landmarks2 is None or len(landmarks1) != len(landmarks2):
        return None  # Landmarks not detected or different numbers of landmarks

    distances = np.linalg.norm(landmarks1 - landmarks2, axis=1)
    mean_distance = np.mean(distances)
    inter_ocular_distance = np.linalg.norm(landmarks1[36] - landmarks1[45])  # Distance between the eyes

    nlmd = mean_distance / inter_ocular_distance
    return nlmd

def get_video_frames(video_path):
    cap = cv2.VideoCapture(video_path)
    frames = []
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frames.append(frame)
    cap.release()
    return frames

def calculate_video_nlmd(video_path1, video_path2):
    frames1 = get_video_frames(video_path1)
    frames2 = get_video_frames(video_path2)
    
    if len(frames1) != len(frames2):
        raise ValueError("Videos do not have the same number of frames")

    total_nlmd = 0
    frame_count = 0
    
    for frame1, frame2 in zip(frames1, frames2):
        landmarks1 = extract_landmarks(frame1)
        landmarks2 = extract_landmarks(frame2)
        nlmd = calculate_nlmd(landmarks1, landmarks2)
        if nlmd is not None:
            total_nlmd += nlmd
            frame_count += 1
    
    if frame_count == 0:
        return None  # No valid frames with detected landmarks

    avg_nlmd = total_nlmd / frame_count
    return avg_nlmd



In [11]:
# Paths to input videos
video1_path = "/home/sysadm/Downloads/emotalkingface/videos/last_time/tr_train_result_neu_soumitra_sir_2_check/neutral_generated.mp4_.mp4"
video2_path = "/home/sysadm/Downloads/emotalkingface/videos/last_time/pre_train_result_neu_soumitra_sir_2_check/neutral_generated.mp4_.mp4"


In [12]:
# Paths to the generated and ground-truth video files
generated_video_path = video1_path
ground_truth_video_path = video2_path

# Calculate average NLMD for the video
avg_nlmd = calculate_video_nlmd(generated_video_path, ground_truth_video_path)
print(f"Average NLMD: {avg_nlmd}")

Average NLMD: 0.014550116600378213


In [ ]:
/home/sysadm/Downloads/emotalkingface/video_result_compare/

# over 50 videos

In [ ]:
#psnr

In [13]:
import cv2
import numpy as np
import os

def calculate_psnr(img1, img2):
    mse = np.mean((img1 - img2) ** 2)
    if mse == 0:
        return float('inf')
    psnr = 20 * np.log10(255.0 / np.sqrt(mse))
    return psnr

def get_video_frames(video_path):
    cap = cv2.VideoCapture(video_path)
    frames = []
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frames.append(frame)
    cap.release()
    return frames

def calculate_video_psnr(video_path1, video_path2):
    frames1 = get_video_frames(video_path1)
    frames2 = get_video_frames(video_path2)
    
    if len(frames1) != len(frames2):
        raise ValueError("Videos do not have the same number of frames")

    total_psnr = 0
    frame_count = len(frames1)
    
    for frame1, frame2 in zip(frames1, frames2):
        psnr = calculate_psnr(frame1, frame2)
        total_psnr += psnr
    
    avg_psnr = total_psnr / frame_count
    return avg_psnr

def calculate_average_psnr_for_folders(base_path):
    folder_names = [name for name in os.listdir(base_path) if os.path.isdir(os.path.join(base_path, name))]
    psnr_values = []
    
    for folder in folder_names:
        folder_path = os.path.join(base_path, folder)
        video_files = [f for f in os.listdir(folder_path) if f.endswith('.mp4')]

        if len(video_files) != 2:
            raise ValueError(f"Folder {folder} does not contain exactly two video files.")
        
        video_path1 = os.path.join(folder_path, video_files[0])
        video_path2 = os.path.join(folder_path, video_files[1])
        
        psnr = calculate_video_psnr(video_path1, video_path2)
        psnr_values.append(psnr)
    
    average_psnr = sum(psnr_values) / len(psnr_values)
    return average_psnr



In [14]:
# Define the base path to the folder containing 50 subfolders
base_path = '/home/sysadm/Downloads/emotalkingface/video_result_compare/'

# Calculate the average PSNR across all folders
average_psnr = calculate_average_psnr_for_folders(base_path)
print(f"Average PSNR for all folders: {average_psnr}")

Average PSNR for all folders: 34.98556464721388


In [ ]:
#SSIM

In [15]:
from skimage.metrics import structural_similarity as ssim
import cv2
import os

def calculate_ssim(img1, img2):
    # Convert the images to grayscale (SSIM is often computed on grayscale images)
    gray_img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
    gray_img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)
    
    # Compute SSIM
    score, _ = ssim(gray_img1, gray_img2, full=True)
    return score
    
def get_video_frames(video_path):
    cap = cv2.VideoCapture(video_path)
    frames = []
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frames.append(frame)
    cap.release()
    return frames

def calculate_video_ssim(video_path1, video_path2):
    frames1 = get_video_frames(video_path1)
    frames2 = get_video_frames(video_path2)
    
    if len(frames1) != len(frames2):
        raise ValueError("Videos do not have the same number of frames")

    total_ssim = 0
    frame_count = len(frames1)
    
    for frame1, frame2 in zip(frames1, frames2):
        ssim_score = calculate_ssim(frame1, frame2)
        total_ssim += ssim_score
    
    avg_ssim = total_ssim / frame_count
    return avg_ssim

def calculate_average_ssim_for_folders(base_path):
    folder_names = [name for name in os.listdir(base_path) if os.path.isdir(os.path.join(base_path, name))]
    ssim_values = []
    
    for folder in folder_names:
        folder_path = os.path.join(base_path, folder)
        video_files = [f for f in os.listdir(folder_path) if f.endswith('.mp4')]

        if len(video_files) != 2:
            raise ValueError(f"Folder {folder} does not contain exactly two video files.")
        
        video_path1 = os.path.join(folder_path, video_files[0])
        video_path2 = os.path.join(folder_path, video_files[1])
        
        ssim_value = calculate_video_ssim(video_path1, video_path2)
        ssim_values.append(ssim_value)
    
    average_ssim = sum(ssim_values) / len(ssim_values)
    return average_ssim



In [16]:
# Define the base path to the folder containing 50 subfolders
base_path = '/home/sysadm/Downloads/emotalkingface/video_result_compare/'

# Calculate the average SSIM across all folders
average_ssim = calculate_average_ssim_for_folders(base_path)
print(f"Average SSIM for all folders: {average_ssim}")

Average SSIM for all folders: 0.8901143821260935


In [ ]:
#NLMD

In [18]:
import dlib
import cv2
import numpy as np

# Load pre-trained face detector and landmark predictor from dlib
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')  # You need to download this file

def extract_landmarks(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    faces = detector(gray)
    if len(faces) == 0:
        return None  # No faces detected
    landmarks = predictor(gray, faces[0])
    landmarks = [(p.x, p.y) for p in landmarks.parts()]
    return np.array(landmarks)

def calculate_nlmd(landmarks1, landmarks2):
    if landmarks1 is None or landmarks2 is None or len(landmarks1) != len(landmarks2):
        return None  # Landmarks not detected or different numbers of landmarks

    distances = np.linalg.norm(landmarks1 - landmarks2, axis=1)
    mean_distance = np.mean(distances)
    inter_ocular_distance = np.linalg.norm(landmarks1[36] - landmarks1[45])  # Distance between the eyes

    nlmd = mean_distance / inter_ocular_distance
    return nlmd

def get_video_frames(video_path):
    cap = cv2.VideoCapture(video_path)
    frames = []
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frames.append(frame)
    cap.release()
    return frames

def calculate_video_nlmd(video_path1, video_path2):
    frames1 = get_video_frames(video_path1)
    frames2 = get_video_frames(video_path2)
    
    if len(frames1) != len(frames2):
        raise ValueError("Videos do not have the same number of frames")

    total_nlmd = 0
    frame_count = 0
    
    for frame1, frame2 in zip(frames1, frames2):
        landmarks1 = extract_landmarks(frame1)
        landmarks2 = extract_landmarks(frame2)
        nlmd = calculate_nlmd(landmarks1, landmarks2)
        if nlmd is not None:
            total_nlmd += nlmd
            frame_count += 1
    
    if frame_count == 0:
        return None  # No valid frames with detected landmarks

    avg_nlmd = total_nlmd / frame_count
    return avg_nlmd

def calculate_average_nlmd_for_folders(base_path):
    folder_names = [name for name in os.listdir(base_path) if os.path.isdir(os.path.join(base_path, name))]
    nlmd_values = []
    
    for folder in folder_names:
        folder_path = os.path.join(base_path, folder)
        video_files = [f for f in os.listdir(folder_path) if f.endswith('.mp4')]

        if len(video_files) != 2:
            raise ValueError(f"Folder {folder} does not contain exactly two video files.")
        
        video_path1 = os.path.join(folder_path, video_files[0])
        video_path2 = os.path.join(folder_path, video_files[1])
        
        nlmd_value = calculate_video_nlmd(video_path1, video_path2)
        nlmd_values.append(nlmd_value)
    
    average_nlmd = sum(nlmd_values) / len(nlmd_values)
    return average_nlmd

In [19]:
# Define the base path to the folder containing 50 subfolders
base_path = '/home/sysadm/Downloads/emotalkingface/video_result_compare/'

# Calculate the average NLMD across all folders
average_ssim = calculate_average_nlmd_for_folders(base_path)
print(f"Average NLMD for all folders: {average_ssim}")

Average NLMD for all folders: 0.034463881245930215
